In [2]:
import pandas as pd

In [77]:
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
ufo_data = pd.read_csv('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv', sep=',')
ufo_df = ufo_data.copy()
bike_data = pd.read_csv('data/citibike-tripdata.csv', sep = ',')
bike_df = bike_data.copy()

In [4]:
customer_df = pd.DataFrame({
            'number': [0, 1, 2, 3, 4],
            'cust_id': [128, 1201, 9832, 4392, 7472],
            'cust_age': [13, 21, 19, 21, 60],
            'cust_sale': [0, 0, 0.2, 0.15, 0.3],
            'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
            'cust_order': [1400, 14142, 900, 1240, 8430]
        })

In [5]:
def delete_columns(df, col=[]):
  for i in col:
    if i not in df:
      return None
  return df.drop(col, axis = 1)

In [7]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})

In [8]:
a = countries_df['population']*1000000/countries_df['square']
b = round(a.mean(), 2)
print(b)

84.93


In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

In [ ]:
years_sold = melb_df['Date'].dt.year
print('Mode year sold:', years_sold.mode()[0])
melb_df['MonthSale'] = melb_df['Date'].dt.month #выделение месяца
melb_df['MonthSale'].value_counts(normalize=True) #частота продаж с выводом в долях
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01')
display(delta_days.dt.days) #чтобы вывести число
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
melb_df = melb_df.drop('YearBuilt', axis=1)

In [11]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5)|(melb_df['WeekdaySale'] == 6)].shape[0]
print(weekend_count)

12822


In [12]:
ufo_df['Time'] = pd.to_datetime(ufo_df['Time'])
years_ufo = ufo_df['Time'].dt.year
print(years_ufo.mode()[0])
ufo_df['Date'] = ufo_df['Time'].dt.date
ufo = ufo_df[ufo_df['State'] == 'NV']['Date'].diff().dt.days.mean()
print(ufo)

1999
68.92932862190813


In [13]:
ufo_df['Date'] = ufo_df['Time'].dt.date
print(ufo_df[ufo_df['State']=='NV']['Date'].diff().dt.days.mean())

68.92932862190813


In [14]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

In [ ]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

In [16]:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


In [ ]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

In [ ]:
def get_weekend(weekday):
    if (weekday == 5)|(weekday == 6):
        return 1
    else:
        return 0
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)

In [30]:
print(round(melb_df[melb_df['Weekend'] == 1]['Price'].mean()))

1081199


In [ ]:
popular_seller = melb_df['SellerG'].value_counts().nlargest(49).index
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')
print((melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()/melb_df[melb_df['SellerG'] == 'other']['Price'].min()))

In [20]:
def get_experience(arg):
    arg_list = arg.split(' ')
    a = arg_list[-1]
    print(a)
    if len(arg_list) > 4:
        return int(arg_list[2])*12 + int(arg_list[4])
    elif a[0:3] == 'мес':
        return int(arg_list[2])
    else:
        return int(arg_list[2])*12

In [ ]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes)     # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
    #игнорируем индексы для создания новых индексов
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца

In [ ]:
print(melb_df['Regionname'].cat.categories) #вывод названий категорий
display(melb_df['Regionname'].cat.codes)

In [ ]:
#переименование категорий
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

In [ ]:
melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)

In [ ]:
popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_seller else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
melb_df.info()

In [ ]:
print(300000 - int(bike_df['start station id'].count()))

In [ ]:
print(bike_df['bikeid'].mode())
print(bike_df['usertype'].value_counts(normalize=True))
print(bike_df['gender'].value_counts())
print(bike_df['birth year'].max())
print(bike_df['end station name'].value_counts())

In [ ]:
bike_df.drop(['start station id', 'end station id'], axis=1)

In [56]:
print(bike_df.shape[1])

12


In [58]:
bike_df['age'] = 2018 - bike_df['birth year']

In [ ]:
bike_df.drop(['birth year'], axis=1, inplace = True)

In [62]:
print(bike_df[bike_df['age'] > 60].shape[0])

11837


In [89]:
bike_df['starttime'] = pd.to_datetime(bike_df['starttime'])
bike_df['stoptime'] = pd.to_datetime(bike_df['stoptime'])
bike_df['trip duration'] = bike_df['stoptime'].dt.hour*60 + bike_df['stoptime'].dt.minute -bike_df['starttime'].dt.hour*60 - bike_df['starttime'].dt.minute
print(bike_df['trip duration'][3])

7


In [92]:
bike_df['weekday'] = bike_df['starttime'].dt.dayofweek
bike_df['weekend'] = bike_df['weekday'].apply(get_weekend)
print(bike_df[bike_df['weekend'] == 1].shape[0])

115135


In [102]:
def get_time(time):
    if 0<= time <=6:
        return 'night'
    elif 6< time <=12:
        return 'morning'
    elif 12< time <=18:
        return 'day'
    else:
        return 'evening'
bike_df['time_of_day'] = bike_df['starttime'].dt.hour.apply(get_time)
print(round(bike_df[bike_df['time_of_day'] == 'day'].shape[0]/bike_df[bike_df['time_of_day'] == 'night'].shape[0]))

9
